## 始めに
全国の地方自治体の名称や緯度経度の情報を、[こちら](https://github.com/code4fukui/localgovjp)からダウンロードしたもの(localgov.csv)と、  
全ての自治体間の直線距離を事前に計算したもの(disance.csv)を使います。

In [ ]:
import pandas as pd
import pulp
import matplotlib.pyplot as plt
import folium
import networkx as nx
from pyproj import  Geod

In [ ]:
# 地方自治体のデータ
localgov_df = pd.read_csv('localgov.csv', usecols=['pref', 'cid', 'city', 'lat', 'lng'])
localgov_df

In [ ]:
# 政令指定都市の場合、「札幌市北区」と「札幌市」が混在しているので、政令指定都市を取り除きます
seirei = ('札幌市', '仙台市', 'さいたま市', '千葉市', '横浜市', '川崎市', '相模原市', '新潟市', '静岡市', '浜松市',
          '名古屋市', '京都市', '大阪市', '堺市', '神戸市', '岡山市', '広島市', '北九州市', '福岡市', '熊本市')
localgov_df = localgov_df[~localgov_df.city.isin(seirei)]

In [ ]:
# 全ての自治体間の距離のデータ(km単位)
df = localgov_df.loc[:, ['cid', 'lat', 'lng']]
df['key'] = 1
df = pd.merge(df, df, on='key', how='outer',suffixes=['1', '2']).drop('key', axis=1)
geod = Geod(ellps='WGS84')
def get_distance(lat1, lng1, lat2, lng2):
    _, _, dist = geod.inv(lng1, lat1, lng2, lat2)
    return dist

df['distance'] = get_distance(df.lat1.tolist(), df.lng1.tolist(), df.lat2.tolist(), df.lng2.tolist())
df.distance = df.distance / 1000
distance_df = df.loc[:, ['cid1', 'cid2', 'distance']]
del df
distance_df


In [ ]:
# 特定の都道府県にデータを絞ってみます
localgov_df = localgov_df[localgov_df.pref=='千葉県']
localgov_df

In [ ]:
# 先ほど絞った都道府県における距離データにします
distance_df = distance_df[(distance_df.cid1.isin(localgov_df.cid) & distance_df.cid2.isin(localgov_df.cid))]
distance_df

In [ ]:
# 最適化で扱いように、それぞれ辞書にします
localgov_dic = localgov_df.set_index('cid').to_dict(orient='index')
localgov_dic[12101]

In [ ]:
distance_dic = distance_df.set_index(['cid1', 'cid2']).to_dict(orient='index')
distance_dic

## 巡回セールスマン問題を厳密解法で解く関数
この時間内では実装について触れません

In [ ]:
def solve_tsp(localgov_dic, distance_dic):

    nodes = list(localgov_dic.keys())
    G = nx.Graph()
    G.add_nodes_from(nodes)
    edges = [(i, j) for i in nodes for j in nodes if i < j]

    # 最適化モデルの定義
    model = pulp.LpProblem('', sense=pulp.LpMinimize)

    # 決定変数x。枝(i, j)を使うか否か。
    x = {(i, j):pulp.LpVariable(f'edge({i}, {j})', cat='Binary') for (i, j) in edges}

    # 制約条件。各都市を必ず1度通る。
    for i in nodes:
        avlbl_edges = [(j, i) for j in nodes if (j, i) in edges] + [(i, j) for j in nodes if (i, j) in edges]
        model += pulp.lpSum( x[edge] for edge in avlbl_edges ) == 2

    # 目的関数。総距離。
    model += pulp.lpSum( distance_dic[i, j]['distance'] * x[i, j] for (i, j) in edges )

    # この条件で解く
    model.solve()

    # 使われる枝
    used_edges = [(i, j) for (i, j) in edges if x[i, j].value() > 0.5]
    G.add_edges_from(used_edges)

    # 部分巡回路除去
    CC = list(nx.connected_components(G))
    while len(CC) > 1:
        for S in CC:
            model += pulp.lpSum( x[i, j] for (i, j) in edges if i in S and j in S ) <= len(S) - 1
        model.solve()

        G.remove_edges_from(used_edges)
        used_edges = [(i, j) for (i, j) in edges if x[i, j].value() > 0.5]
        G.add_edges_from(used_edges)
        CC = list(nx.connected_components(G))

    return model.objective.value(), x



In [ ]:
# 実際に解く
TourLength, x = solve_tsp(localgov_dic, distance_dic)
TourLength

## 結果の可視化
matplotlibを使って、最適なルートを描画します

In [ ]:
# 使われる枝
edges = [(i, j) for (i, j) in x if x[i, j].value()==1]
# 体裁を整える
fig = plt.figure(figsize=(10, 10))
plt.xlim([localgov_df.lng.min() - 0.05, localgov_df.lng.max() + 0.05])
plt.ylim([localgov_df.lat.min() - 0.05, localgov_df.lat.max() + 0.05])
plt.xlabel('longitude')
plt.ylabel('latitude')
plt.title('Tour')
# 都市
for i in localgov_dic:
    plt.scatter(localgov_dic[i]['lng'], localgov_dic[i]['lat'], color='dimgray')
# ルートを引く
for (city_i, city_j) in edges:
    plt.plot(
        [localgov_dic[city_i]['lng'], localgov_dic[city_j]['lng']]
        , [localgov_dic[city_i]['lat'], localgov_dic[city_j]['lat']]
        , color='royalblue'
    )
plt.show()

Foliumを使って、地図の上にプロットしてみます

In [ ]:
# 下地を用意する。地図の中心の座標や、最初の拡大レベルを指定する
tour_map = folium.Map(location=[localgov_df.lat.mean(), localgov_df.lng.mean()], tiles='cartodbpositron', zoom_start=9)
# 使われる枝の部分に線を引く
for (city_i, city_j) in edges:
    # 始点と終点の緯度経度
    both_ends = [
        [localgov_dic[city_i]['lat'], localgov_dic[city_i]['lng']]
        , [localgov_dic[city_j]['lat'], localgov_dic[city_j]['lng']]
    ]
    # 2点間の線を引く
    folium.vector_layers.PolyLine(
        locations=both_ends
    ).add_to(tour_map)
# 各地方自治体の位置にマーカーを置く
for city in localgov_dic:
    # 1つ1つのマーカー
    folium.Circle(
        location=[localgov_dic[city]['lat'], localgov_dic[city]['lng']]
        , popup=localgov_dic[city]['city']
        , radius=800
        , fill=True
        , fill_color='#706A73'
        , color='#706A73'
    ).add_to(tour_map)

tour_map